# 이상패턴으로  판정한 200개 host에 대한 집중 분석

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import rc

In [2]:
df = pd.read_csv('../dataset/track1.csv') # 데이터 프레임 초기화
df.Timestamp = pd.to_datetime(df.Timestamp) 
df = df[df['Status'].astype(str).str.isnumeric()] # 이상치 제거
df.head()

,Timestamp,Method,Protocol,Status,Referer,Path,Host,UA,Payload,Bytes
0,2020-08-24 10:00:18,GET,HTTP/1.1,200,"""-""",/1000/wp-includes/,101.224.32.28,Mozilla/5.0 (Macintosh; Intel Mac OS X x.y; rv...,-,6226.0
1,2020-08-24 10:00:18,GET,HTTP/1.1,200,"""-""",/1000/wp-login/,101.224.32.28,Mozilla/5.0 (Macintosh; Intel Mac OS X x.y; rv...,-,6226.0
2,2020-08-24 10:00:18,GET,HTTP/1.1,200,"""-""",/1000/wp-register/,101.224.32.28,Mozilla/5.0 (Macintosh; Intel Mac OS X x.y; rv...,-,6232.0
3,2020-08-24 10:00:18,GET,HTTP/1.1,200,"""-""",/1000/wp/,101.224.32.28,Mozilla/5.0 (Macintosh; Intel Mac OS X x.y; rv...,-,6248.0
4,2020-08-24 10:00:19,GET,HTTP/1.1,200,"""-""",/1000/wp-content/,101.224.32.28,Mozilla/5.0 (Macintosh; Intel Mac OS X x.y; rv...,-,6224.0


In [4]:
patternHost = ['189.102.137.171',
 '166.146.28.109',
 '93.139.7.167',
 '215.82.66.58',
 '62.126.184.49',
 '154.183.104.68',
 '192.217.106.4',
 '33.45.176.173',
 '209.163.210.98',
 '158.7.116.147',
 '157.29.174.233',
 '218.186.207.158',
 '155.110.66.204',
 '113.163.220.14',
 '96.220.20.38',
 '56.215.113.246',
 '253.72.61.176',
 '174.212.113.195',
 '181.208.169.183',
 '231.162.115.137',
 '128.60.55.30',
 '99.199.46.146',
 '230.91.143.108',
 '206.52.153.133',
 '41.235.176.98',
 '110.39.9.172',
 '194.48.94.171',
 '49.2.124.41',
 '13.112.117.158',
 '231.229.17.216',
 '180.113.58.104',
 '53.70.235.161',
 '9.27.108.183',
 '108.59.199.146',
 '58.33.250.234',
 '144.218.200.213',
 '77.51.208.6',
 '119.165.69.87',
 '223.7.119.122',
 '36.206.166.96',
 '207.44.179.175',
 '193.102.46.159',
 '118.194.121.61',
 '185.204.235.94',
 '87.232.221.120',
 '251.215.60.237',
 '122.236.50.249',
 '42.154.105.206',
 '249.221.133.56',
 '31.241.156.71',
 '15.120.133.168',
 '68.246.216.126',
 '241.22.8.121',
 '236.162.53.8',
 '122.75.246.242',
 '210.159.169.113',
 '34.69.121.11',
 '149.231.211.148',
 '114.240.227.197',
 '75.98.230.167',
 '38.140.103.156',
 '111.120.134.184',
 '248.245.36.197',
 '140.22.209.246',
 '176.120.230.123',
 '73.25.144.98',
 '218.59.66.15',
 '244.192.204.214',
 '138.189.245.218',
 '22.160.80.50',
 '228.50.148.130',
 '45.43.242.153',
 '231.233.83.222',
 '247.83.158.223',
 '164.138.189.108',
 '38.219.8.203',
 '59.223.59.238',
 '5.157.107.195',
 '68.70.177.148',
 '52.235.119.115',
 '139.182.73.5',
 '177.208.153.113',
 '155.47.151.70',
 '25.232.211.103',
 '63.237.133.245',
 '37.64.245.73',
 '32.149.239.191',
 '89.118.222.166',
 '57.198.254.244',
 '184.13.27.188',
 '69.7.97.191',
 '88.125.247.149',
 '83.183.235.143',
 '84.229.129.163',
 '160.137.37.1',
 '198.208.191.234',
 '237.20.113.120',
 '144.74.30.34',
 '226.93.172.27',
 '85.176.102.221',
 '99.64.144.149',
 '129.197.168.17',
 '108.200.175.182',
 '179.109.202.163',
 '85.180.217.131',
 '167.145.154.40',
 '118.52.128.24',
 '223.55.122.183',
 '3.168.105.10',
 '134.253.9.38',
 '120.207.123.148',
 '248.208.120.149',
 '134.149.35.190',
 '64.50.31.61',
 '175.214.223.16',
 '235.160.25.129',
 '84.83.171.161',
 '9.112.183.31',
 '55.198.38.0',
 '31.62.244.218',
 '201.116.192.99',
 '232.143.134.156',
 '219.126.10.96',
 '62.66.235.25',
 '42.88.127.114',
 '238.60.95.5',
 '34.121.125.108',
 '219.240.45.95',
 '42.101.37.130',
 '242.162.51.166',
 '115.130.124.88',
 '41.121.130.0',
 '6.210.151.21',
 '155.101.146.48',
 '73.52.126.133',
 '145.104.81.213',
 '30.11.55.212',
 '129.236.75.8',
 '132.134.0.43',
 '203.165.187.206',
 '36.227.196.135',
 '35.146.170.79',
 '223.199.240.6',
 '183.209.224.212',
 '81.226.230.91',
 '55.185.6.120',
 '137.3.55.226',
 '123.252.177.154',
 '178.245.100.17',
 '219.229.163.141',
 '125.197.213.96',
 '73.159.164.232',
 '12.118.38.74',
 '39.112.75.139',
 '77.28.42.72',
 '10.161.50.95',
 '187.82.32.231',
 '187.19.60.109',
 '0.242.182.71',
 '236.117.113.88',
 '198.215.148.228',
 '106.254.46.141',
 '57.40.245.32',
 '109.126.152.174',
 '96.55.243.39',
 '147.245.128.177',
 '79.219.245.90',
 '236.79.11.24',
 '120.59.68.21',
 '97.157.219.191',
 '48.156.186.164',
 '115.112.105.145',
 '34.110.106.4',
 '100.227.232.242',
 '169.161.230.151',
 '34.235.217.86',
 '238.39.228.156',
 '212.254.82.60',
 '113.138.175.149',
 '124.236.155.199',
 '189.59.12.228',
 '219.40.100.241',
 '179.233.243.212',
 '120.126.24.186',
 '152.188.88.120',
 '85.49.64.128',
 '66.112.27.100',
 '155.2.137.36',
 '130.99.247.192',
 '241.166.70.82',
 '83.202.72.192',
 '141.127.187.88',
 '86.75.151.184',
 '77.13.148.197',
 '244.216.242.67',
 '205.254.127.151',
 '127.202.178.51',
 '89.133.124.168',
 '53.119.161.113',
 '240.174.206.196']

In [7]:
patternHost = '|'.join(patternHost)
patternHost

'189.102.137.171|166.146.28.109|93.139.7.167|215.82.66.58|62.126.184.49|154.183.104.68|192.217.106.4|33.45.176.173|209.163.210.98|158.7.116.147|157.29.174.233|218.186.207.158|155.110.66.204|113.163.220.14|96.220.20.38|56.215.113.246|253.72.61.176|174.212.113.195|181.208.169.183|231.162.115.137|128.60.55.30|99.199.46.146|230.91.143.108|206.52.153.133|41.235.176.98|110.39.9.172|194.48.94.171|49.2.124.41|13.112.117.158|231.229.17.216|180.113.58.104|53.70.235.161|9.27.108.183|108.59.199.146|58.33.250.234|144.218.200.213|77.51.208.6|119.165.69.87|223.7.119.122|36.206.166.96|207.44.179.175|193.102.46.159|118.194.121.61|185.204.235.94|87.232.221.120|251.215.60.237|122.236.50.249|42.154.105.206|249.221.133.56|31.241.156.71|15.120.133.168|68.246.216.126|241.22.8.121|236.162.53.8|122.75.246.242|210.159.169.113|34.69.121.11|149.231.211.148|114.240.227.197|75.98.230.167|38.140.103.156|111.120.134.184|248.245.36.197|140.22.209.246|176.120.230.123|73.25.144.98|218.59.66.15|244.192.204.214|138.189.24

In [11]:
patternDf = df[df['Host'].str.contains(patternHost, na=False)]
patternDf.head()

,Timestamp,Method,Protocol,Status,Referer,Path,Host,UA,Payload,Bytes
63958,2020-08-25 12:11:43,GET,HTTP/1.1,200,"""http://wp.hotpost.kr/""",/?p=4659,193.102.46.159,Mozilla/5.0 (Windows NT 6.3; Win64; x64) Apple...,-,24249.0
63959,2020-08-25 12:11:43,GET,HTTP/1.1,200,"""http://wp.hotpost.kr/""",/?m=20200815,249.221.133.56,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,-,44094.0
63960,2020-08-25 12:11:43,GET,HTTP/1.1,200,"""http://wp.hotpost.kr/""",/?p=694,31.241.156.71,Mozilla/5.0 (X11; Ubuntu; Linux i686 on x86_64...,-,23199.0
63961,2020-08-25 12:11:43,GET,HTTP/1.1,200,"""http://wp.hotpost.kr/""",/?p=3012,36.206.166.96,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2)...,-,32605.0
63963,2020-08-25 12:11:43,GET,HTTP/1.1,200,"""http://wp.hotpost.kr/""",/?p=3421,215.82.66.58,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...,-,39041.0


In [12]:
patternDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 879089 entries, 63958 to 1530762
Data columns (total 10 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   Timestamp  879089 non-null  datetime64[ns]
 1   Method     879089 non-null  object        
 2   Protocol   879089 non-null  object        
 3   Status     879089 non-null  object        
 4   Referer    879089 non-null  object        
 5   Path       879089 non-null  object        
 6   Host       879089 non-null  object        
 7   UA         879089 non-null  object        
 8   Payload    879089 non-null  object        
 9   Bytes      879089 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(8)
memory usage: 73.8+ MB


In [29]:
patternHostGrouped = patternDf.groupby('Host')
funcAvgDifftime = lambda g:np.mean(g['Timestamp'].diff().dt.seconds.div(1, fill_value=0))
funcStdDifftime = lambda g:np.std(g['Timestamp'].diff().dt.seconds.div(1, fill_value=0))
avgDifftime = patternHostGrouped.apply(funcAvgDifftime)
stdDifftime = patternHostGrouped.apply(funcStdDifftime)

In [30]:
patternTimeDiff = pd.concat([(patternHostGrouped['Timestamp'].count().rename('Count')), 
                      avgDifftime.rename('mean_diff_time'), 
                      stdDifftime.rename('std_diff_time')], axis=1)

In [31]:
patternTimeDiff = patternTimeDiff.sort_values(by='Count', ascending=False)
patternTimeDiff

,Count,mean_diff_time,std_diff_time
Host,,,
99.64.144.149,5044,2.653648,25.246343
129.197.168.17,4956,2.701170,25.444516
108.200.175.182,4952,2.703554,25.495030
15.120.133.168,4932,2.776156,26.721912
68.246.216.126,4916,2.784174,26.824862
...,...,...,...
205.254.127.151,3163,2.993045,26.926463
127.202.178.51,3161,2.993357,26.759742
89.133.124.168,3153,3.002854,26.843138


In [18]:
attackHost = ['101.224.32.28', 
             '231.211.11.16', '137.8.46.133', '100.200.156.222', '112.112.181.134', '188.45.31.10', '188.45.31.20', '188.45.31.30', '188.45.31.40', '14.135.56.110', '14.135.56.120', '14.135.56.130', '14.135.56.140']

In [19]:
attackHost = '|'.join(attackHost)

In [20]:
attackDf = df[df['Host'].str.contains(attackHost, na=False)]
attackDf.head()

,Timestamp,Method,Protocol,Status,Referer,Path,Host,UA,Payload,Bytes
0,2020-08-24 10:00:18,GET,HTTP/1.1,200,"""-""",/1000/wp-includes/,101.224.32.28,Mozilla/5.0 (Macintosh; Intel Mac OS X x.y; rv...,-,6226.0
1,2020-08-24 10:00:18,GET,HTTP/1.1,200,"""-""",/1000/wp-login/,101.224.32.28,Mozilla/5.0 (Macintosh; Intel Mac OS X x.y; rv...,-,6226.0
2,2020-08-24 10:00:18,GET,HTTP/1.1,200,"""-""",/1000/wp-register/,101.224.32.28,Mozilla/5.0 (Macintosh; Intel Mac OS X x.y; rv...,-,6232.0
3,2020-08-24 10:00:18,GET,HTTP/1.1,200,"""-""",/1000/wp/,101.224.32.28,Mozilla/5.0 (Macintosh; Intel Mac OS X x.y; rv...,-,6248.0
4,2020-08-24 10:00:19,GET,HTTP/1.1,200,"""-""",/1000/wp-content/,101.224.32.28,Mozilla/5.0 (Macintosh; Intel Mac OS X x.y; rv...,-,6224.0


In [21]:
attackDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 201959 entries, 0 to 1258826
Data columns (total 10 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   Timestamp  201959 non-null  datetime64[ns]
 1   Method     201959 non-null  object        
 2   Protocol   201959 non-null  object        
 3   Status     201959 non-null  object        
 4   Referer    201959 non-null  object        
 5   Path       201959 non-null  object        
 6   Host       201959 non-null  object        
 7   UA         201959 non-null  object        
 8   Payload    201959 non-null  object        
 9   Bytes      201633 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(8)
memory usage: 16.9+ MB


In [26]:
attackHostGrouped = attackDf.groupby('Host')
funcAvgDifftime = lambda g:np.mean(g['Timestamp'].diff().dt.seconds.div(1, fill_value=0))
funcStdDifftime = lambda g:np.std(g['Timestamp'].diff().dt.seconds.div(1, fill_value=0))
avgDifftime = attackHostGrouped.apply(funcAvgDifftime)
stdDifftime = attackHostGrouped.apply(funcStdDifftime)

In [27]:
attackTimeDiff = pd.concat([(attackHostGrouped['Timestamp'].count().rename('Count')), 
                      avgDifftime.rename('mean_diff_time'), 
                      stdDifftime.rename('std_diff_time')], axis=1)

In [28]:
attackTimeDiff = attackTimeDiff.sort_values(by='Count', ascending=False)
attackTimeDiff

,Count,mean_diff_time,std_diff_time
Host,,,
101.224.32.28,27757,10.058472,723.741209
112.112.181.134,25404,0.405960,5.499969
188.45.31.30,25071,0.449922,28.720695
188.45.31.20,23562,0.361599,33.545618
231.211.11.16,22238,1.197815,37.024191
188.45.31.10,19068,0.204531,4.274526
188.45.31.40,15720,0.221374,7.772323
100.200.156.222,9980,1.749699,75.213130
137.8.46.133,7911,2.439262,21.872340


In [34]:
anormalHost = patternHost +'|'+ attackHost
anormalHost

'189.102.137.171|166.146.28.109|93.139.7.167|215.82.66.58|62.126.184.49|154.183.104.68|192.217.106.4|33.45.176.173|209.163.210.98|158.7.116.147|157.29.174.233|218.186.207.158|155.110.66.204|113.163.220.14|96.220.20.38|56.215.113.246|253.72.61.176|174.212.113.195|181.208.169.183|231.162.115.137|128.60.55.30|99.199.46.146|230.91.143.108|206.52.153.133|41.235.176.98|110.39.9.172|194.48.94.171|49.2.124.41|13.112.117.158|231.229.17.216|180.113.58.104|53.70.235.161|9.27.108.183|108.59.199.146|58.33.250.234|144.218.200.213|77.51.208.6|119.165.69.87|223.7.119.122|36.206.166.96|207.44.179.175|193.102.46.159|118.194.121.61|185.204.235.94|87.232.221.120|251.215.60.237|122.236.50.249|42.154.105.206|249.221.133.56|31.241.156.71|15.120.133.168|68.246.216.126|241.22.8.121|236.162.53.8|122.75.246.242|210.159.169.113|34.69.121.11|149.231.211.148|114.240.227.197|75.98.230.167|38.140.103.156|111.120.134.184|248.245.36.197|140.22.209.246|176.120.230.123|73.25.144.98|218.59.66.15|244.192.204.214|138.189.24

In [35]:
normalDf = df[~df['Host'].str.contains(anormalHost, na=False)]
normalDf.head()

,Timestamp,Method,Protocol,Status,Referer,Path,Host,UA,Payload,Bytes
9,2020-08-24 10:01:16,GET,HTTP/1.1,200,"""http://wp.hotpost.kr/""",/?p=594,25.235.106.214,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,-,24322.0
10,2020-08-24 10:01:28,GET,HTTP/1.1,200,"""http://wp.hotpost.kr/""",/?p=3659,210.128.88.9,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,-,34403.0
12,2020-08-24 10:01:37,GET,HTTP/1.1,200,"""http://wp.hotpost.kr/""",/?p=6705,149.226.49.1,Mozilla/5.0 (X11; OpenBSD amd64; rv:28.0) Geck...,-,24921.0
13,2020-08-24 10:01:39,GET,HTTP/1.1,200,"""http://wp.hotpost.kr/""",/?p=4085,149.148.142.101,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,-,29736.0
14,2020-08-24 10:01:49,GET,HTTP/1.1,200,"""http://wp.hotpost.kr/""",/?p=5013,6.243.131.98,Mozilla/5.0 (Windows NT 6.4; WOW64) AppleWebKi...,-,24609.0


In [37]:
normalDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 455960 entries, 9 to 1550249
Data columns (total 10 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   Timestamp  455960 non-null  datetime64[ns]
 1   Method     455960 non-null  object        
 2   Protocol   455960 non-null  object        
 3   Status     455960 non-null  object        
 4   Referer    455475 non-null  object        
 5   Path       455960 non-null  object        
 6   Host       451375 non-null  object        
 7   UA         455960 non-null  object        
 8   Payload    455960 non-null  object        
 9   Bytes      455960 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(8)
memory usage: 38.3+ MB


In [39]:
normalHostGrouped = normalDf.groupby('Host')
funcAvgDifftime = lambda g:np.mean(g['Timestamp'].diff().dt.seconds.div(1, fill_value=0))
funcStdDifftime = lambda g:np.std(g['Timestamp'].diff().dt.seconds.div(1, fill_value=0))
avgDifftime = normalHostGrouped.apply(funcAvgDifftime)
stdDifftime = normalHostGrouped.apply(funcStdDifftime)

In [40]:
normalTimeDiff = pd.concat([(normalHostGrouped['Timestamp'].count().rename('Count')), 
                      avgDifftime.rename('mean_diff_time'), 
                      stdDifftime.rename('std_diff_time')], axis=1)

In [51]:
normalTimeDiff = normalTimeDiff.sort_values(by='mean_diff_time', ascending=True)
normalTimeDiff[:50]

,Count,mean_diff_time,std_diff_time
Host,,,
27.57.40.99,1,0.000000,0.000000
204.103.240.191,1,0.000000,0.000000
147.26.39.207,1,0.000000,0.000000
170.54.6.180,1,0.000000,0.000000
183.143.153.71,1,0.000000,0.000000
105.3.128.26,1,0.000000,0.000000
22.9.85.84,1,0.000000,0.000000
136.129.246.45,1,0.000000,0.000000
199.242.38.6,1,0.000000,0.000000


In [75]:
attackTimeDiff.describe()

,Count,mean_diff_time,std_diff_time
count,13.000000,13.000000,13.000000
mean,15535.307692,1.495029,80.538444
std,8610.544905,2.660657,194.738840
min,5871.000000,0.204531,3.718888
25%,7090.000000,0.361599,7.772323
50%,15720.000000,0.449922,21.872340
75%,23562.000000,1.197815,37.024191
max,27757.000000,10.058472,723.741209


In [76]:
normalTimeDiff.describe()

,Count,mean_diff_time,std_diff_time
count,3780.000000,3780.000000,3780.000000
mean,119.411376,1272.750826,2125.446876
std,157.216242,1548.954950,3430.224737
min,1.000000,0.000000,0.000000
25%,14.000000,79.153319,139.353942
50%,28.000000,915.223684,1012.833171
75%,180.000000,1815.464154,2073.351632
max,551.000000,28413.666667,26062.909742


In [78]:
patternTimeDiff.describe()

,Count,mean_diff_time,std_diff_time
count,200.000000,200.000000,200.000000
mean,4395.445000,2.847140,26.525528
std,669.779171,0.065649,0.517052
min,3129.000000,2.653648,25.246343
25%,4273.750000,2.799488,26.068675
50%,4751.500000,2.839343,26.565985
75%,4817.250000,2.887382,26.911487
max,5044.000000,3.025567,27.613596
